<a href="https://colab.research.google.com/github/sujitojha1/EVA4/blob/rev8/S15/EVA4_S15_Solution_DenseDepth_step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EVA4 Session 15 Assignment - DepthMap & Mask Prediction

## 1. Loading the dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir 'fg_bg'
!mkdir 'mask'
!mkdir 'depth_map'

In [3]:
%%time
!cp -r '/content/drive/My Drive/EVA4/S15/fg_bg.zip' fg_bg/train.zip 
!cp -r '/content/drive/My Drive/EVA4/S15/mask.zip' mask/train.zip 
!cp -r '/content/drive/My Drive/EVA4/S15/depth_map.zip' depth_map/train.zip 

!unzip -o -qq fg_bg/train.zip -d fg_bg/train/
!unzip -o -qq mask/train.zip -d mask/train/
!unzip -o -qq depth_map/train.zip -d depth_map/train/

CPU times: user 996 ms, sys: 163 ms, total: 1.16 s
Wall time: 9min 21s


In [4]:
import os
import glob
import numpy as np

from PIL import Image
print(os.getcwd())
file_list = [filename for filename in glob.iglob('./mask/*.*')]

/content


In [0]:
%%time
from torch.utils import data
from torchvision import datasets, transforms
dataset = datasets.ImageFolder('fg_bg', transform=transforms.Compose([transforms.ToTensor()]))

loader = data.DataLoader(dataset,
                         batch_size=100,
                         num_workers=4,
                         shuffle=False)

mean = 0.0
for images, _ in loader:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(loader.dataset)

var = 0.0
for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(loader.dataset)*224*224))

In [0]:
import torch
std = torch.sqrt(var / (len(loader.dataset)*224*224))

In [9]:
mean

tensor([0.5704, 0.5221, 0.4675])

In [10]:
std

tensor([0.2504, 0.2552, 0.2709])

In [11]:
%%time
from torch.utils import data
from torchvision import datasets, transforms
dataset = datasets.ImageFolder('mask', transform=transforms.Compose([transforms.ToTensor()]))

loader = data.DataLoader(dataset,
                         batch_size=100,
                         num_workers=4,
                         shuffle=False)

mean = 0.0
for images, _ in loader:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(loader.dataset)

var = 0.0
for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(loader.dataset)*224*224))

CPU times: user 5min 25s, sys: 2min 17s, total: 7min 43s
Wall time: 11min 17s


In [12]:
print(mean)
print(std)

tensor([0.0345, 0.0345, 0.0345])
tensor([0.1796, 0.1796, 0.1796])


In [13]:
%%time
from torch.utils import data
from torchvision import datasets, transforms
dataset = datasets.ImageFolder('depth_map', transform=transforms.Compose([transforms.ToTensor()]))

loader = data.DataLoader(dataset,
                         batch_size=100,
                         num_workers=4,
                         shuffle=False)

mean = 0.0
for images, _ in loader:
    batch_samples = images.size(0) 
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
mean = mean / len(loader.dataset)

var = 0.0
for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    var += ((images - mean.unsqueeze(1))**2).sum([0,2])
std = torch.sqrt(var / (len(loader.dataset)*112*112))
print(mean)
print(std)

tensor([0.3448, 0.3448, 0.3448])
tensor([0.1401, 0.1401, 0.1401])
CPU times: user 1min 47s, sys: 43.4 s, total: 2min 30s
Wall time: 4min 27s
